In [3]:

# Prints versions for key libraries with robust fallback logic so packages
# without __version__ (e.g., linformer) still report a version from metadata.

import sys
import importlib

try:
    import importlib.metadata as imd  # Py3.8+
except Exception:  # pragma: no cover
    imd = None

packages = [
    ("torch", "torch"),
    ("numpy", "numpy"),
    ("pandas", "pandas"),
    ("scikit-learn", "sklearn"),  # display name, import name
    ("tqdm", "tqdm"),
    ("tiktoken", "tiktoken"),
    ("linformer", "linformer"),
    ("psutil", "psutil"),
    ("matplotlib", "matplotlib"),
]

def resolve_version(display_name: str, import_name: str):
    """Return best-effort version string for a package.
    1. Try imported module.__version__ (with sklearn special case)
    2. Try importlib.metadata.version on several candidate distribution names
    3. Fallback to 'UNKNOWN'
    """
    mod = None
    try:
        mod = importlib.import_module(import_name)
    except Exception:
        return "NOT INSTALLED"

    # Direct attribute
    ver = getattr(mod, "__version__", None)
    if ver is None and import_name == "sklearn":  # sklearn exposes __version__ at package root
        try:
            from sklearn import __version__ as skv  # type: ignore
            ver = skv
        except Exception:
            pass

    # Metadata fallback
    if (ver is None or ver in ("0+unknown", "unknown", "None")) and imd is not None:
        candidates = {
            import_name,
            display_name,
            display_name.replace('-', '_'),
            import_name.replace('_', '-'),
        }
        for cand in candidates:
            try:
                ver = imd.version(cand)
                if ver:
                    break
            except Exception:
                continue

    return ver if ver else "UNKNOWN"

print("Library Versions (pin these in requirements.txt):")
max_name = max(len(p[0]) for p in packages)
rows = []
for display_name, import_name in packages:
    ver = resolve_version(display_name, import_name)
    rows.append((display_name, ver))
    print(f"{display_name.ljust(max_name)} : {ver}")

print(f"Python{' '.ljust(max_name-6)} : {sys.version.split()[0]}")

# Optional: also show a pip-installable line block for easy copy/paste
pip_lines = [f"{name}=={ver}" for name, ver in rows if ver not in ("UNKNOWN", "NOT INSTALLED")]
if pip_lines:
    print("\n# Copy/paste for requirements.txt:")
    for l in pip_lines:
        print(l)
else:
    print("\n(No resolvable package versions to list.)")


Library Versions (pin these in requirements.txt):
torch        : 2.7.1+cu128
numpy        : 2.3.1
pandas       : 2.3.1
scikit-learn : 1.7.0
tqdm         : 4.67.1
tiktoken     : 0.9.0
linformer    : 0.2.3
psutil       : 7.0.0
matplotlib   : 3.10.3
Python       : 3.12.11

# Copy/paste for requirements.txt:
torch==2.7.1+cu128
numpy==2.3.1
pandas==2.3.1
scikit-learn==1.7.0
tqdm==4.67.1
tiktoken==0.9.0
linformer==0.2.3
psutil==7.0.0
matplotlib==3.10.3
